In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
from sklearn.metrics import adjusted_mutual_info_score


from hisel import select
from hisel.select import FeatureType, HSICSelector as Selector

In [ ]:
k = 5
n = 1000
d = 10

In [ ]:
x0 = np.random.randint(k, size=(n, 1))
x1 = np.random.randint(k, size=(n, 1))
ms = np.random.randint(low=2, high=20, size = d-2)
others = [np.random.choice(m, size=(n, 1)) for m in ms]
all_ = np.concatenate(
    [x0, x1] + others,
    axis=1
)
y = np.asarray(x0 == x1, dtype=int) # k + x0 - x1 # np.asarray(x0 == x1, dtype=int)
permuter =  np.random.permutation(np.eye(d, dtype=int).T).T
x = np.array(all_ @ permuter, dtype=int)
expected_features = [np.argmax(permuter[0, :]), np.argmax(permuter[1, :])]

In [ ]:
assert np.all(x[:, expected_features[0]] == x0[:, 0])
assert np.all(x[:, expected_features[1]] == x1[:, 0])

In [ ]:
sns.scatterplot(x = x0[:, 0] - x1[:, 0], y = y[:, 0])

In [ ]:
xdf = pd.DataFrame(x, columns = [f'x{i}' for i in range(d)])
ydf = pd.DataFrame(y, columns = ['y0'])

In [ ]:
ksgselection, mis = select.ksgmi(xdf, ydf, threshold=0.01)

In [ ]:
selector = Selector(x, y, xfeattype=FeatureType.DISCR, yfeattype=FeatureType.DISCR)

In [ ]:
batch_size = n // 10
minibatch_size = 200
number_of_epochs = 3
threshold = .0
device = None # run on CPU

In [ ]:
selection = selector.select(
    number_of_features=2,
    batch_size=batch_size,
    minibatch_size=minibatch_size,
    number_of_epochs=number_of_epochs,
    device=device
)

In [ ]:
autoselection = selector.autoselect(
    threshold=threshold,
    batch_size=batch_size,
    minibatch_size=minibatch_size,
    number_of_epochs=number_of_epochs,
    device=device
)

In [ ]:
expected_features

In [ ]:
selection

In [ ]:
autoselection

In [ ]:
mis

In [ ]:
def onedimlabel(x):
    assert x.ndim == 2
    ns = np.amax(x, axis=0)
    res = np.array(x[:, 0], copy=True)
    m = 1
    for i in range(1, x.shape[1]):
        m *= max(1, ns[i-1])
        res += (1+m) * x[:, i]
    return res

In [ ]:
miscores = {subset: 
           adjusted_mutual_info_score(onedimlabel(x[:, list(subset)]), y[:, 0])
            for l in range(1, 3)
            for subset in itertools.combinations(list(range(d)), l)
            
}

In [ ]:
miscores

In [ ]:
s = (0,1)
mi = 0
for k, v in miscores.items():
    if v > mi:
        s = k
        mi = v

In [ ]:
s

In [ ]:
miscores[s]

In [ ]:
from hisel import hsic

In [ ]:
n * n * hsic.hsic_b(
    x[:, s],
    y
)